## Financial Statement Crawling

In [14]:
import dart_fss as dart

api_key='998bebe5acc23a9e8321f35d52181529440718e8'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

# 삼성전자 검색
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0]

# 2012년부터 연간 연결재무제표 불러오기
fs = samsung.extract_fs(bgn_de='20120101', report_tp='quarter')

# 재무제표 검색 결과를 엑셀파일로 저장 ( 기본저장위치: 실행폴더/fsdata )
fs.save('samsumg_test')

Annual reports:   0%|          | 0/10 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/9 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/18 [00:00<?, ?report/s]

ValueError: No engine for filetype: ''

In [17]:
fs.save(filename='samsung_test.xlsx')

'/Users/klee30810/Documents/Career/Finance/halto/fsdata/samsung_test.xlsx'

In [9]:
# 연결재무상태표
df_bs = fs['bs'] # 또는 df = fs[0] 또는 df = fs.show('bs')
# 연결재무상태표 추출에 사용된 Label 정보
labels_bs = fs.labels['bs']

# 연결손익계산서
df_is = fs['is'] # 또는 df = fs[1] 또는 df = fs.show('is')
# 연결손익계산서 추출에 사용된 Label 정보
labels_is = fs.labels['is']

# 연결포괄손익계산서
df_ci = fs['cis'] # 또는 df = fs[2] 또는 df = fs.show('cis')
# 연결포괄손익계산서 추출에 사용된 Label 정보
labels_ci = fs.labels['cis']

# 현금흐름표
df_cf = fs['cf'] # 또는 df = fs[3] 또는 df = fs.show('cf')
# 현금흐름표 추출에 사용된 Label 정보
labels_cf = fs.labels['cf']


In [22]:
fs.show('fs',show_class=True, show_concept=True)

KeyError: 'fs'

## stock price crawling

In [60]:
import pandas as pd

stock_code = pd.read_excel('210509_kospi.xlsx',sheet_name='Sheet1', 
                          converters={'단축코드': str})
stock_code

/Users/klee30810/miniconda3/envs/halto/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,표준코드,단축코드,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,액면가,상장주식수
0,KR7095570008,095570,AJ네트웍스보통주,AJ네트웍스,"AJ Networks Co.,Ltd.",2015/08/21,KOSPI,주권,NaN,보통주,1000,46822295
1,KR7006840003,006840,AK홀딩스보통주,AK홀딩스,"AK Holdings, Inc.",1999/08/11,KOSPI,주권,NaN,보통주,5000,13247561
2,KR7282330000,282330,BGF리테일보통주,BGF리테일,BGF Retail,2017/12/08,KOSPI,주권,NaN,보통주,1000,17283906
3,KR7027410000,027410,BGF보통주,BGF,BGF,2014/05/19,KOSPI,주권,NaN,보통주,1000,95716791
4,KR7138930003,138930,BNK금융지주보통주,BNK금융지주,BNK Financial Group Inc.,2011/03/30,KOSPI,주권,NaN,보통주,5000,325935246
...,...,...,...,...,...,...,...,...,...,...,...,...
917,KR7069260008,069260,휴켐스보통주,휴켐스,HuchemsFineChemicalCorporation,2002/10/07,KOSPI,주권,NaN,보통주,1000,40878588
918,KR7000541003,000545,흥국화재해상보험1우선주,흥국화재우,HeungkukFire&MarineIns(1P),1990/03/20,KOSPI,주권,NaN,구형우선주,5000,768000
919,KR7000542001,000547,흥국화재해상보험2우선주(신형),흥국화재2우B,HeungkukFire&MarineIns(2PB),1999/08/09,KOSPI,주권,NaN,신형우선주,5000,153600
920,KR7000540005,000540,흥국화재해상보험보통주,흥국화재,HeungkukFire&MarineInsurance,1974/12/05,KOSPI,주권,NaN,보통주,5000,64242645


In [61]:
stock_code=stock_code[['단축코드', '한글 종목약명']]
stock_code = stock_code.loc[lambda stock_code: stock_code['한글 종목약명'] == '현대차']

In [62]:
stock_code

,단축코드,한글 종목약명
881,005380,현대차


In [63]:
# 네이버 금융 10년 주가 정보 수집 코드
import requests
from bs4 import BeautifulSoup
import xmltodict
import json

def get_daily_stock_price(stockCode, name, count):
    
    url = f'https://fchart.stock.naver.com/sise.nhn?symbol={stockCode}&timeframe=day&count={count}&requestType=0'
    
    rs = requests.get(url)
    dt = xmltodict.parse(rs.text)
    js = json.dumps(dt, indent=4)
    js = json.loads(js)
    
    data = pd.json_normalize(js['protocol']['chartdata']['item'])
    df = data['@data'].str.split('|', expand=True)
    df.columns = ['date', 'open', 'high', 'low', 'close', 'Volume']
    df['name'] = str(name) # 받아온 기업명 추가
    
    return df

In [64]:
tmp = []

for index, row in stock_code.iterrows():
    tmp.append(get_daily_stock_price(row['단축코드'], row['한글 종목약명'], '2500'))

tmp

[          date    open    high     low   close   Volume name
 0     20110317  178000  181000  175000  178500   841527  현대차
 1     20110318  179500  183500  178500  179500   915781  현대차
 2     20110321  179000  182000  178500  181000   432274  현대차
 3     20110322  182000  189000  181000  189000  1280694  현대차
 4     20110323  189000  192500  186000  190500  1363379  현대차
 ...        ...     ...     ...     ...     ...      ...  ...
 2495  20210430  216000  216500  211000  212000  1168115  현대차
 2496  20210503  212000  222500  211000  218000  1189242  현대차
 2497  20210504  220000  224500  217500  223500   953550  현대차
 2498  20210506  223000  225000  221000  223500   645594  현대차
 2499  20210507  223500  227000  221000  224000   598369  현대차
 
 [2500 rows x 7 columns]]

In [65]:
df_final = pd.concat(tmp, ignore_index=True)
df_final

,date,open,high,low,close,Volume,name
0,20110317,178000,181000,175000,178500,841527,현대차
1,20110318,179500,183500,178500,179500,915781,현대차
2,20110321,179000,182000,178500,181000,432274,현대차
3,20110322,182000,189000,181000,189000,1280694,현대차
4,20110323,189000,192500,186000,190500,1363379,현대차
...,...,...,...,...,...,...,...
2495,20210430,216000,216500,211000,212000,1168115,현대차
2496,20210503,212000,222500,211000,218000,1189242,현대차
2497,20210504,220000,224500,217500,223500,953550,현대차
2498,20210506,223000,225000,221000,223500,645594,현대차


In [66]:
df_final.to_csv('HDMstockprice.csv', encoding='euc-kr')

## 시총 가져오기

In [171]:
import requests
from bs4 import BeautifulSoup

def get_market_price(pagecount):
    url = f'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page={pagecount}'
    
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    
    stock_list = soup.find("table", attrs={"class": "type_2"}).find("tbody").find_all("tr")
    
    tmp = []
    for stock in stock_list:
        if len(stock)>1:
            tmp.append(stock.get_text().split())

    
    tmp = pd.DataFrame(tmp).iloc[:,[1,6]]
    tmp = tmp.values.tolist()
    
    return tmp


In [184]:
pagecount = 9

tmp = []

for index in range(1,pagecount):
    index = str(index)
    tmp = tmp + get_market_price(index)
    

In [195]:
df

,회사,시가총액(억)
0,삼성전자,"4,889,252"
1,SK하이닉스,"942,763"
2,LG화학,"648,744"
3,삼성전자우,"615,519"
4,NAVER,"592,991"
...,...,...
395,남선알미늄,"4,016"
396,제일파마홀딩스,"4,001"
397,한솔제지,"3,998"
398,삼부토건,"3,996"


In [199]:
df = pd.DataFrame(tmp, columns=['회사','시가총액(억)'])
#df['시가총액(억)']=pd.to_numeric(df['시가총액(억)'])
df.to_excel('KospiMarketPrice.xlsx', encoding='euc-kr')


In [74]:
stock_head = soup.find("thead").find_all("th")
data_head = [head.get_text() for head in stock_head]

print(data_head)

['N', '종목명', '현재가', '전일비', '등락률', '액면가', '시가총액', '상장주식수', '외국인비율', '거래량', 'PER', 'ROE', '토론실']


## ETF 일봉 크롤링

In [126]:
import pandas as pd

etf_code = pd.read_excel('210509_koretf.xlsx',sheet_name='Sheet1', 
                          converters={'단축코드': str})
etf_code

/Users/klee30810/miniconda3/envs/halto/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,표준코드,단축코드,한글종목명,한글종목약명,영문종목명,상장일,기초지수명,지수산출기관,추적배수,복제방법,기초시장분류,기초자산분류,상장좌수,운용사,CU수량,총보수,과세유형
0,KR7292340007,292340,DB 마이티 200커버드콜ATM레버리지증권상장지수투자신탁[주식-파생형],마이티 200커버드콜ATM레버리지,DB Mighty KOSPI200 Covered Call ATM Leverage ETF,2018/03/20,코스피 200 커버드콜 ATM 지수,KRX,2X 레버리지 (2),실물,국내,주식,600000,디비자산운용,100000,0.510,배당소득세(보유기간과세)
1,KR7159800002,159800,DB마이티K100증권상장지수투자신탁(주식),마이티 코스피100,DB Mighty K100 ETF,2012/07/05,코스피 100,KRX,일반 (1),실물,국내,주식,280000,디비자산운용,40000,0.390,비과세
2,KR7361580004,361580,KB KBSTAR 200 Total Return증권상장지수투자신탁(주식),KBSTAR 200TR,KB KBSTAR 200 Total Return ETF,2020/08/21,코스피 200 TR,KRX,일반 (1),실물,국내,주식,6150000,케이비자산운용,50000,0.012,배당소득세(보유기간과세)
3,KR7285000006,285000,KB KBSTAR 200IT증권상장지수투자신탁(주식),KBSTAR 200IT,KB KBSTAR 200 Information Technology ETF,2017/12/08,코스피 200 정보기술,KRX,일반 (1),실물,국내,주식,720000,케이비자산운용,20000,0.190,비과세
4,KR7287300008,287300,KB KBSTAR 200건설증권상장지수투자신탁(주식),KBSTAR 200건설,KB KBSTAR 200 Constructions ETF,2017/12/22,코스피 200 건설,KRX,일반 (1),실물,국내,주식,200000,케이비자산운용,20000,0.190,비과세
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,KR7213630007,213630,한화ARIRANG DJ US Select Dividend 증권상장지수투자신탁(주식-...,ARIRANG 미국다우존스고배당주(합성 H),Hanwha ARIRANG SYNTH-DJ US Select Dividend ETF(H),2015/01/26,Dow Jones U.S Select Dividend Index(시장가격지수),S&P,일반 (1),합성,해외,주식,1530000,한화자산운용,10000,0.400,배당소득세(보유기간과세)
465,KR7189400005,189400,한화ARIRANG MSCI AC World 증권상장지수투자신탁(주식-파생형)(합성 H),ARIRANG 글로벌MSCI(합성 H),Hanwha ARIRANG SYNTH-MSCI AC World ETF(H),2013/12/10,MSCI ACWI Index,MSCI,일반 (1),합성,해외,주식,300000,한화자산운용,30000,0.400,배당소득세(보유기간과세)
466,KR7195970009,195970,한화ARIRANG MSCI EAFE 증권상장지수투자신탁(주식-파생형)(합성 H),ARIRANG 선진국MSCI(합성 H),Hanwha ARIRANG SYNTH-MSCI EAFE ETF(H),2014/05/13,MSCI EAFE Index,MSCI,일반 (1),합성,해외,주식,520000,한화자산운용,20000,0.500,배당소득세(보유기간과세)
467,KR7195980008,195980,한화ARIRANG MSCI Emerging Markets 증권상장지수투자신탁(주식-...,ARIRANG 신흥국MSCI(합성 H),Hanwha ARIRANG SYNTH-MSCI Emerging Markets ETF(H),2014/05/13,MSCI EM Index,MSCI,일반 (1),합성,해외,주식,5560000,한화자산운용,20000,0.500,배당소득세(보유기간과세)


In [127]:
etf_code = etf_code[['단축코드', '한글종목약명']]
#etf_code = etf_code.loc[lambda etf_code: etf_code['한글종목약명'] == 'KODEX 200']
etf_code

,단축코드,한글종목약명
0,292340,마이티 200커버드콜ATM레버리지
1,159800,마이티 코스피100
2,361580,KBSTAR 200TR
3,285000,KBSTAR 200IT
4,287300,KBSTAR 200건설
...,...,...
464,213630,ARIRANG 미국다우존스고배당주(합성 H)
465,189400,ARIRANG 글로벌MSCI(합성 H)
466,195970,ARIRANG 선진국MSCI(합성 H)
467,195980,ARIRANG 신흥국MSCI(합성 H)


In [71]:
# 네이버 etf 주가 정보 수집 코드
import requests
from bs4 import BeautifulSoup
import xmltodict
import json

def get_daily_etf_price(etfCode, name, start, end):
    
    url = f'https://fchart.stock.naver.com/siseJson.naver?symbol={etfCode}&requestType=1&startTime={start}&endTime={end}&timeframe=day'
    
    rs = requests.get(url)
    dt = xmltodict.parse(rs.text)
    js = json.dumps(dt, indent=4)
    js = json.loads(js)
    
    data = pd.json_normalize(js['protocol']['chartdata']['item'])
    df = data['@data'].str.split('|', expand=True)
    df.columns = ['date', 'open', 'high', 'low', 'close', 'Volume', 'foreign']
    df['name'] = str(name) # 받아온 기업명 추가
    
    return df

get_daily_etf_price(etf_code['단축코드'], etf_code['한글종목약명'],'20200302', '20210507')


[['날짜', '시가', '고가', '저가', '종가', '거래량', '외국인소진율']]



ExpatError: syntax error: line 2, column 0

In [74]:
etf_list


NameError: name 'etf_list' is not defined

In [148]:
import xmltodict
import datetime
import requests
from bs4 import BeautifulSoup
from itertools import islice
from datetime import datetime
import numpy as np

def get_etf_info(etfCode, start, end):
    ## etf 주가 정보
    url = f'https://api.finance.naver.com/siseJson.naver?symbol={etfCode}&requestType=1&startTime={start}&endTime={end}&timeframe=day'
    rs = requests.get(url)

    soup = BeautifulSoup(rs.text, 'html.parser')
    soup = soup.get_text().split()

    # 필요 없는 문자 지우기
    for i in range(0, len(soup)):
        soup[i] = soup[i].replace('\t','').replace('\n','').replace('\'','').replace(',','').replace('[','').replace(']','').replace('\"','')

    # 두 날짜 차이 구하기
    time1 = 20200304
    time2 = 20210509
    time1 = datetime((int)(time1/10000), (int)(time1%10000/100), (int)(time1%100),0,0,0)
    time2 = datetime((int)(time2/10000), (int)(time2%10000/100), (int)(time2%100))
    financial_timediff = (int)((time2 - time1).days/30) * 20
    financial_timediff = 293

    df_size = np.zeros(((int)(len(soup)/7),7))
    df = pd.DataFrame(df_size)
    soup = pd.DataFrame(soup)
  
    # 2D array
    for i in range(0, (int)(len(soup)/7)):
        for j in range(0, 7):
            df.iloc[i,j] = soup.iloc[7*i+j]
            
    return df

df = get_etf_info('213630', '20200104', '20210510')
print(df)

            0      1      2      3      4      5       6
0          날짜     시가     고가     저가     종가    거래량  외국인소진율
1    20200106  13820  13860  13775  13845  14484     0.0
2    20200107  13855  13880  13770  13800   2269     0.0
3    20200108  13840  13840  13750  13790   2955     0.0
4    20200109  13870  13900  13835  13900   1359     0.0
..        ...    ...    ...    ...    ...    ...     ...
329  20210503  15580  15740  15580  15735  94662     0.0
330  20210504  15800  15910  15730  15910  60161     0.0
331  20210506  15905  15950  15850  15900  61944     0.0
332  20210507  16040  16190  16030  16190  85398     0.0
333  20210510  16130  16200  16065  16185  56928     0.0

[334 rows x 7 columns]


In [84]:
soup = pd.DataFrame(temp)
#df['시가총액(억)']=pd.to_numeric(df['시가총액(억)'])
soup.to_excel('etftest.xlsx', encoding='euc-kr', index=False, header=False)

## 1,3,6,12M 수익률

In [266]:
# etf
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

url = f'https://finance.naver.com/item/main.nhn?code=153130'
rs = requests.get(url)

soup = BeautifulSoup(rs.text, 'html.parser')
ret_soup = str(soup.find("table", attrs={"summary":"1개월 수익률 정보"}).find("tbody").find_all("tr"))

# 필요 없는 태그 지우기
ret_soup = re.sub('<.+?>','',ret_soup,0).strip()
ret_soup = ret_soup.replace('\n','').replace('\t','').replace(',','').replace('수익률','').replace('[','').replace(']','').split()

# 수익률 형태 맞추기
# soup = pd.DataFrame(soup)
# ret_dict = {"1개월 수익률":soup.iloc[1],"3개월 수익률":soup.iloc[3],"6개월 수익률":soup.iloc[5],"1년 수익률":soup.iloc[7]}
# ret_list = pd.DataFrame(ret_dict)

# 시가총액
agg_soup = str(soup.find("table", attrs={"summary":"시가총액 정보"})).find("tr", attrs={"class":"strong"})
#agg_soup = agg_soup.replace('\n','').replace('\t','').replace(',','').replace('[','').replace(']','').split()
agg_soup

# ret_list


TypeError: find() takes no keyword arguments

In [291]:
# stock
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

url = f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=006840&cn='
rs = requests.get(url)

soup = BeautifulSoup(rs.text, 'html.parser')
soup = str(soup.find("table", attrs={"class":"gHead"}).find("tbody").find_all("tr"))#.find_all("td",attrs={"class":"num noline-bottom"}))

# 필요 없는 태그 지우기
soup = re.sub('<.+?>','',soup,0).strip()
soup = soup.replace('\n','').replace('\t','').replace(',','').replace('수익률','').replace('[','').replace(']','').replace('(1M/3M/6M/1Y)','').replace('%/','%').replace('억원','').split()

# 시가총액 추출
agg_price = np.array(pd.DataFrame(soup).iloc[18])

# 모멘텀 형태 맞추기
ret_soup = np.array(pd.DataFrame(soup).iloc[27:])
ret_dict = {"시가총액(억원)":agg_price,"1개월 수익률":ret_soup[0],"3개월 수익률":ret_soup[1],"6개월 수익률":ret_soup[2],"1년 수익률":ret_soup[3]}
ret_list = pd.DataFrame(ret_dict)

ret_list


,시가총액(억원),1개월 수익률,3개월 수익률,6개월 수익률,1년 수익률
0,4438,+13.56%,+2.76%,+65.84%,+45.34%


In [1]:
# stock
import pandas as pd
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

stock_code = pd.read_excel('210509_kospi.xlsx',sheet_name='Sheet1', 
                          converters={'단축코드': str})
stock_code = stock_code[['단축코드', '한글 종목약명']]


def get_stock_momentum(etfCode, name):
    url = f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={etfCode}&cn='
    rs = requests.get(url)

    soup = BeautifulSoup(rs.text, 'html.parser')
    soup = str(soup.find("table", attrs={"class":"gHead"}).find("tbody").find_all("tr"))#.find_all("td",attrs={"class":"num noline-bottom"}))

    # 필요 없는 태그 지우기
    soup = re.sub('<.+?>','',soup,0).strip()
    soup = soup.replace('\n','').replace('\t','').replace(',','').replace('수익률','').replace('[','').replace(']','').replace('(1M/3M/6M/1Y)','').replace('%/','%').replace('억원','').split()

    # 시가총액 추출
    agg_price = np.array(pd.DataFrame(soup).iloc[18])

    # 모멘텀 형태 맞추기
    ret_soup = np.array(pd.DataFrame(soup).iloc[27:])
    ret_dict = {"기업명":name,"시가총액(억원)":agg_price,"1개월 수익률":ret_soup[0],"3개월 수익률":ret_soup[1],"6개월 수익률":ret_soup[2],"1년 수익률":ret_soup[3]}
    ret_list = pd.DataFrame(ret_dict)

    return ret_list


temp_df = get_stock_momentum(stock_code['단축코드'][1], stock_code['한글 종목약명'][1])
for i in range(2, len(stock_code)):
    temp_df = pd.concat([temp_df, get_stock_momentum(stock_code['단축코드'][i], stock_code['한글 종목약명'][i])])

temp_df

/Users/klee30810/miniconda3/envs/halto/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NameError: name 'np' is not defined

In [355]:
temp_df.to_excel('stock_momentum_test.xlsx', encoding='euc-kr', index=False)

## 가치 지표 크롤링

- 유동비율 :
- 부채비율 : 
- 이자보상비율 :
- 영업이익 증가율 :
- EPS 증가율 :
- 영업이익률 :
- ROA
- ROE 
- ROIC
- 순운전자본회전율 :
- PER :
- PCR : 
- PSR : 
- PBR :
- EV/EBITDA :
- FCFF : 


https://www.queeneconomy.com/15

In [3]:
import pandas as pd
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

# Financial Ratio Soup
ratio_url = f'http://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'
ratio_rs = requests.get(ratio_url)
ratio_soup = BeautifulSoup(ratio_rs.text, 'html.parser')

# Investment Index Soup
index_url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
index_rs = requests.get(index_url)
index_soup = BeautifulSoup(index_rs.text, 'html.parser')


''' 재무 비율 '''
'''----------------- 안정성 비율 -------------------------'''
# 유동 비율
liquid_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_1","class":"rwf acd_dep_start_close"}).find_all("td"))
liquid_ratio = re.sub('<.+?>','',liquid_ratio,0).strip()
liquid_ratio = liquid_ratio.replace('[','').replace(']','').replace(',','').split()

# 부채비율
liab_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_3","class":"rwf acd_dep_start_close"}).find_all("td"))
liab_ratio = re.sub('<.+?>','',liab_ratio,0).strip()
liab_ratio = liab_ratio.replace('[','').replace(']','').replace(',','').split()

# 이자보상배율
int_compen_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_6","class":"rwf acd_dep_start_close"}).find_all("td"))
int_compen_ratio = re.sub('<.+?>','',int_compen_ratio,0).strip()
int_compen_ratio = int_compen_ratio.replace('[','').replace(']','').replace(',','').split()

'''----------------- 성장성 비율 -------------------------'''
# 영업이익 증가율
opincome_growth_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_10","class":"rwf acd_dep_start_close"}).find_all("td"))
opincome_growth_ratio = re.sub('<.+?>','',opincome_growth_ratio,0).strip()
opincome_growth_ratio = opincome_growth_ratio.replace('[','').replace(']','').replace(',','').split()

# EPS 증가율
eps_growth_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_12","class":"rwf acd_dep_start_close"}).find_all("td"))
eps_growth_ratio = re.sub('<.+?>','',eps_growth_ratio,0).strip()
eps_growth_ratio = eps_growth_ratio.replace('[','').replace(']','').replace(',','').split()

'''----------------- 수익률 비율 -------------------------'''
# 영업이익률
opincome_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_15","class":"rwf acd_dep_start_close"}).find_all("td"))
opincome_ratio = re.sub('<.+?>','',opincome_ratio,0).strip()
opincome_ratio = opincome_ratio.replace('[','').replace(']','').replace(',','').split()

# ROA
roa = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_17","class":"rwf acd_dep_start_close"}).find_all("td"))
roa = re.sub('<.+?>','',roa,0).strip()
roa = roa.replace('[','').replace(']','').replace(',','').split()

# ROE
roe = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_18","class":"rwf acd_dep_start_close"}).find_all("td"))
roe = re.sub('<.+?>','',roe,0).strip()
roe = roe.replace('[','').replace(']','').replace(',','').split()

# ROIC
roic = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_19","class":"rwf acd_dep_start_close"}).find_all("td"))
roic = re.sub('<.+?>','',roic,0).strip()
roic = roic.replace('[','').replace(']','').replace(',','').split()

'''----------------- 활동성 비율 -------------------------'''
# 순운전자본 회전율
workcapital_turnover = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_23","class":"rwf acd_dep_start_close"}).find_all("td"))
workcapital_turnover = re.sub('<.+?>','',workcapital_turnover,0).strip()
workcapital_turnover = workcapital_turnover.replace('[','').replace(']','').replace(',','').split()


''' 투자 지표 '''
'''----------------- Multiples -------------------------'''
# PER
per = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_9","class":"rwf acd_dep_start_close"}).find_all("td"))
per = re.sub('<.+?>','',per,0).strip()
per = per.replace('[','').replace(']','').replace(',','').split()

# PCR
pcr = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_10","class":"rwf acd_dep_start_close"}).find_all("td"))
pcr = re.sub('<.+?>','',pcr,0).strip()
pcr = pcr.replace('[','').replace(']','').replace(',','').split()

# PSR
psr = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_11","class":"rwf acd_dep_start_close"}).find_all("td"))
psr = re.sub('<.+?>','',psr,0).strip()
psr = psr.replace('[','').replace(']','').replace(',','').split()

# PBR
pbr = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_12","class":"rwf acd_dep_start_close"}).find_all("td"))
pbr = re.sub('<.+?>','',pbr,0).strip()
pbr = pbr.replace('[','').replace(']','').replace(',','').split()

# EV/EBITDA
ev_ebitda = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_14","class":"rwf acd_dep_start_close"}).find_all("td"))
ev_ebitda = re.sub('<.+?>','',ev_ebitda,0).strip()
ev_ebitda = ev_ebitda.replace('[','').replace(']','').replace(',','').split()

'''----------------- FCF -------------------------'''
# FCFF
fcff = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find_all("tr", attrs={"class":"rwf"}))#.find_all("td"))
fcff = re.sub('<.+?>','',fcff,0).strip()
fcff = fcff.replace('[','').replace(']','').replace(',','').split()
fcff = fcff[-5:]

## aggregating
temp_agg = liquid_ratio + liab_ratio + int_compen_ratio + opincome_growth_ratio + eps_growth_ratio + opincome_ratio + roa + roe + roic + workcapital_turnover + per + pcr + psr + pbr + ev_ebitda + fcff
temp_agg = pd.DataFrame(temp_agg).transpose()

df_agg = pd.DataFrame(np.zeros([16,5]))
for i in range(0, 16):
    for j in range(0, 5):
        df_agg.iloc[i,j] = temp_agg[5*i+j]
df_agg.columns = ["2016","2017","2018","2019","2020"]
df_agg.rename(index = {0:"유동비율", 1:"부채비율", 2:"이자보상배율", 3:"영업이익증가율",4:"EPS증가율", 5:"영업이익률", 6:"ROA", 7:"ROE", 8:"ROIC", 9:"순운전자본회전율", 10:"PER", 11:"PCR", 12:"PSR", 13:"PBR", 14:"EV/EBITDA", 15:"FCFF"}, inplace=True)
df_agg = df_agg.transpose()
df_agg.iloc[4,:]

유동비율          262.2
부채비율           37.1
이자보상배율         61.7
영업이익증가율        29.6
EPS증가율         21.3
영업이익률          15.2
ROA             7.2
ROE            10.0
ROIC           17.2
순운전자본회전율       29.5
PER           21.09
PCR            9.75
PSR            2.32
PBR            2.06
EV/EBITDA      8.33
FCFF         320372
Name: 2020, dtype: object

In [8]:
import pandas as pd
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def get_stock_ratio(etfCode, name):
    # 시가총액 크롤링
    url = f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={etfCode}&cn='
    rs = requests.get(url)

    soup = BeautifulSoup(rs.text, 'html.parser')
    soup = str(soup.find("table", attrs={"class":"gHead"}).find("tbody").find_all("tr"))#.find_all("td",attrs={"class":"num noline-bottom"}))

    # 필요 없는 태그 지우기
    soup = re.sub('<.+?>','',soup,0).strip()
    soup = soup.replace('\n','').replace('\t','').replace(',','').replace('수익률','').replace('[','').replace(']','').replace('(1M/3M/6M/1Y)','').replace('%/','%').replace('억원','').split()

    # 시가총액 추출
    agg_price = np.array(pd.DataFrame(soup).iloc[18])

    # Financial Ratio Soup
    ratio_url = f'http://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{etfCode}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'
    ratio_rs = requests.get(ratio_url)
    ratio_soup = BeautifulSoup(ratio_rs.text, 'html.parser')

    # Investment Index Soup
    index_url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{etfCode}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
    index_rs = requests.get(index_url)
    index_soup = BeautifulSoup(index_rs.text, 'html.parser')


    ''' 재무 비율 '''
    '''----------------- 안정성 비율 -------------------------'''
    # 유동 비율
    liquid_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_1","class":"rwf acd_dep_start_close"}).find_all("td"))
    liquid_ratio = re.sub('<.+?>','',liquid_ratio,0).strip()
    liquid_ratio = liquid_ratio.replace('[','').replace(']','').replace(',','').split()

    # 부채비율
    liab_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_3","class":"rwf acd_dep_start_close"}).find_all("td"))
    liab_ratio = re.sub('<.+?>','',liab_ratio,0).strip()
    liab_ratio = liab_ratio.replace('[','').replace(']','').replace(',','').split()

    # 이자보상배율
    int_compen_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_6","class":"rwf acd_dep_start_close"}).find_all("td"))
    int_compen_ratio = re.sub('<.+?>','',int_compen_ratio,0).strip()
    int_compen_ratio = int_compen_ratio.replace('[','').replace(']','').replace(',','').split()

    '''----------------- 성장성 비율 -------------------------'''
    # 영업이익 증가율
    opincome_growth_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_10","class":"rwf acd_dep_start_close"}).find_all("td"))
    opincome_growth_ratio = re.sub('<.+?>','',opincome_growth_ratio,0).strip()
    opincome_growth_ratio = opincome_growth_ratio.replace('[','').replace(']','').replace(',','').split()

    # EPS 증가율
    eps_growth_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_12","class":"rwf acd_dep_start_close"}).find_all("td"))
    eps_growth_ratio = re.sub('<.+?>','',eps_growth_ratio,0).strip()
    eps_growth_ratio = eps_growth_ratio.replace('[','').replace(']','').replace(',','').split()

    '''----------------- 수익률 비율 -------------------------'''
    # 영업이익률
    opincome_ratio = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_15","class":"rwf acd_dep_start_close"}).find_all("td"))
    opincome_ratio = re.sub('<.+?>','',opincome_ratio,0).strip()
    opincome_ratio = opincome_ratio.replace('[','').replace(']','').replace(',','').split()

    # ROA
    roa = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_17","class":"rwf acd_dep_start_close"}).find_all("td"))
    roa = re.sub('<.+?>','',roa,0).strip()
    roa = roa.replace('[','').replace(']','').replace(',','').split()

    # ROE
    roe = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_18","class":"rwf acd_dep_start_close"}).find_all("td"))
    roe = re.sub('<.+?>','',roe,0).strip()
    roe = roe.replace('[','').replace(']','').replace(',','').split()

    # ROIC
    roic = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_19","class":"rwf acd_dep_start_close"}).find_all("td"))
    roic = re.sub('<.+?>','',roic,0).strip()
    roic = roic.replace('[','').replace(']','').replace(',','').split()

    '''----------------- 활동성 비율 -------------------------'''
    # 순운전자본 회전율
    workcapital_turnover = str(ratio_soup.find("div", attrs={"class":"ul_col2wrap pd_t25"}).find("tr", attrs={"id":"p_grid1_23","class":"rwf acd_dep_start_close"}).find_all("td"))
    workcapital_turnover = re.sub('<.+?>','',workcapital_turnover,0).strip()
    workcapital_turnover = workcapital_turnover.replace('[','').replace(']','').replace(',','').split()


    ''' 투자 지표 '''
    '''----------------- Multiples -------------------------'''
    # PER
    per = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_9","class":"rwf acd_dep_start_close"}).find_all("td"))
    per = re.sub('<.+?>','',per,0).strip()
    per = per.replace('[','').replace(']','').replace(',','').split()

    # PCR
    pcr = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_10","class":"rwf acd_dep_start_close"}).find_all("td"))
    pcr = re.sub('<.+?>','',pcr,0).strip()
    pcr = pcr.replace('[','').replace(']','').replace(',','').split()

    # PSR
    psr = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_11","class":"rwf acd_dep_start_close"}).find_all("td"))
    psr = re.sub('<.+?>','',psr,0).strip()
    psr = psr.replace('[','').replace(']','').replace(',','').split()

    # PBR
    pbr = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_12","class":"rwf acd_dep_start_close"}).find_all("td"))
    pbr = re.sub('<.+?>','',pbr,0).strip()
    pbr = pbr.replace('[','').replace(']','').replace(',','').split()

    # EV/EBITDA
    ev_ebitda = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find("tr", attrs={"id":"p_grid1_14","class":"rwf acd_dep_start_close"}).find_all("td"))
    ev_ebitda = re.sub('<.+?>','',ev_ebitda,0).strip()
    ev_ebitda = ev_ebitda.replace('[','').replace(']','').replace(',','').split()

    '''----------------- FCF -------------------------'''
    # FCFF
    fcff = str(index_soup.find("table", attrs={"class":"us_table_ty1 h_fix zigbg_no"}).find_all("tr", attrs={"class":"rwf"}))#.find_all("td"))
    fcff = re.sub('<.+?>','',fcff,0).strip()
    fcff = fcff.replace('[','').replace(']','').replace(',','').split()
    fcff = fcff[-5:]

    ## aggregating
    temp_agg = liquid_ratio + liab_ratio + int_compen_ratio + opincome_growth_ratio + eps_growth_ratio + opincome_ratio + roa + roe + roic + workcapital_turnover + per + pcr + psr + pbr + ev_ebitda + fcff
    temp_agg = pd.DataFrame(temp_agg).transpose()
    
    df_agg = pd.DataFrame(np.zeros([16,5]))
    if temp_agg.shape[1] != 80: 
        print("Not enough data, may unavailable in 2016!")
        return df_agg;

    for i in range(0, 16):
        for j in range(0, 5):
#             if temp_agg[5*i+j].empty:
#                 df_agg.iloc[i,j] = "N/A"
#             else:
                df_agg.iloc[i,j] = temp_agg[5*i+j]
            
    df_agg.columns = ["2016","2017","2018","2019","2020"]
    df_agg.rename(index = {0:"유동비율", 1:"부채비율", 2:"이자보상배율", 3:"영업이익증가율",4:"EPS증가율", 5:"영업이익률", 6:"ROA", 7:"ROE", 8:"ROIC", 9:"순운전자본회전율", 10:"PER", 11:"PCR", 12:"PSR", 13:"PBR", 14:"EV/EBITDA", 15:"FCFF"}, inplace=True)
    df_agg = df_agg.transpose()
    df_agg = df_agg.iloc[4,:]
    df_agg["기업 명"] = name
    df_agg["시가총액 (억원)"] = agg_price

    return df_agg

print(pd.DataFrame(get_stock_ratio('027410', 'BGF')))

             2020
유동비율        671.0
부채비율          7.9
이자보상배율       42.2
영업이익증가율     -44.5
EPS증가율        1.2
영업이익률         9.5
ROA           1.1
ROE           1.8
ROIC          9.5
순운전자본회전율      N/A
PER         17.20
PCR         11.56
PSR          3.03
PBR          0.31
EV/EBITDA   17.73
FCFF         1699
기업 명          BGF
시가총액 (억원)  [6633]


In [10]:
from tqdm import tqdm
import pdb

print(stock_code['한글 종목약명'][1])
temp_df = get_stock_ratio(stock_code['단축코드'][1], stock_code['한글 종목약명'][1])
for i in tqdm(range(2, len(stock_code))):
    print(stock_code['한글 종목약명'][i])
    temp_df = pd.concat([temp_df, get_stock_ratio(stock_code['단축코드'][i], stock_code['한글 종목약명'][i])])


temp_df

AK홀딩스


  0%|          | 0/920 [00:00<?, ?it/s]

BGF리테일


  0%|          | 1/920 [00:00<11:49,  1.30it/s]

Not enough data, may unavailable in 2016!
BGF


  0%|          | 2/920 [00:01<10:45,  1.42it/s]

BNK금융지주


  0%|          | 2/920 [00:01<14:47,  1.03it/s]


AttributeError: 'NoneType' object has no attribute 'find_all'

In [514]:
%debug(get_stock_ratio)

UsageError: Line magic function `%debug(get_stock_ratio)` not found.


In [500]:
stock_code

,단축코드,한글 종목약명
0,095570,AJ네트웍스
1,006840,AK홀딩스
2,282330,BGF리테일
3,027410,BGF
4,138930,BNK금융지주
...,...,...
917,069260,휴켐스
918,000545,흥국화재우
919,000547,흥국화재2우B
920,000540,흥국화재
